
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: introducción a los árboles de decisión</h3></font>
***

El objetivo de este *notebook* es familiarizarnos con los principales conceptos de los árboles de decisión, así como su validación y características generales.

In [1]:
# ------------------------------------------------------------------------------
# Librerías
# ------------------------------------------------------------------------------
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import export_text

from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

## Para mostrar gráficos en línea con las celdas del Jupyter
%matplotlib inline  


## Problema de clasificación

En primer lugar, generaremos un conjunto de datos sintético con la finalidad de poder jugar con diferentes nubes de puntos y entender los principales conceptos. La generación la haremos con la función <a href="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html#sklearn.datasets.make_blobs">make_globs</a> de <a href="https://scikit-learn.org/stable/index.html">scikit-learn</a>, que nos permite generar nubes de puntos con una variabilidad determinada. Los centros de las nubes son elegidos de forma aleatoria por esta función.

In [ ]:
## -- Parámetros con los que jugar -- ##

## Nubes de puntos
Nblobs_class0 = 3 
Nblobs_class1 = 3

## Número de muestras en cada clase
nsamples_class0 = 300 
nsamples_class1 = 300

## Variables explicativas
nattributes = 2

## Desviación estándar ~ tamaño
blob_width_01 = 5
blob_width_02 = 5

## Nombres
class_names = ['clase 0', 'clase 1']
attributes_names = ['variable 0', 'variable 1']

## Semilla. Para asegurar reproducción. Cambiando estos parámetros se obtienen nubes de puntos distintos
random_state_01 = 21
random_state_02 = 23

X_class0, _ = make_blobs(random_state = random_state_01,
                         n_samples = nsamples_class0,
                         n_features = nattributes,
                         centers = Nblobs_class0,
                         cluster_std = blob_width_01)

X_class1, _ = make_blobs(random_state = random_state_02,
                         n_samples = nsamples_class1,
                         n_features = nattributes,
                         centers = Nblobs_class1, 
                         cluster_std = blob_width_02)

X = np.vstack((X_class0, X_class1))
y = np.zeros(nsamples_class0 + nsamples_class1)
y[nsamples_class0:] = 1

plt.figure(figsize=(7, 5))
plt.scatter(X[y==0, 0], X[y==0, 1], color = 'r', label=class_names[0])
plt.scatter(X[y==1, 0], X[y==1, 1], color = 'b', label=class_names[1])
plt.grid()
plt.xlabel(attributes_names[0])
plt.ylabel(attributes_names[1])
plt.title('Datos sintéticos')
plt.legend(loc=2)
plt.show()


Una vez tenemos el conjunto de datos generado, vamos a realizar la partición de los datos en:

- **Train**. Conjunto que usaremos para realizar el entrenamiento del modelo.
- **Test**. Conjunto que nos permitirá validar la capacidad predictiva del modelo a partir del cálculo de determiandas métricas. Por lo tanto, este conjunto no lo ve el modelo en ningún momento del aprendizaje.

Recordar que en problemas de clasificación existen diferentes métricas de validación: curva ROC, matriz de confusión para medir precisión y recall, entre otras.


In [17]:
## Partición en train y test
testsize = 0.3 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=5)

## La semilla nos permite asegurar la reproducción de la partición

Ahora vamos a entrenar un árbol de decisión a través del módulo <a href="http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier">DecisionTreeClassifier</a>. Los parámetros más importantes son los siguientes:

- **criterion:** criterio para la partición de los nodos, es decir, función que mide la calidad de la partición. Podría ser 'gini' o 'entropy'.

- **max_depth:** profundidad máxima del árbol.

Como hemos visto en las diapositivas, las muestras que llegan a un nodo del árbol se usan para calcular estadísticas que estimen la calidad de las posibles particiones o preguntas en ese nodo (Gini o Entropía). Además, también se usan para calcular estimaciones relacionadas con la clase que se va a predecir en caso de que no se realicen más divisiones. Por lo tanto, el número de ejemplos debería ser lo suficientemente grande para asegurar la calidad de las estimaciones. Entonces, surge la necesidad de fijar unos requisitos mínimos sobre la cantidad de ejemplos que podemos controlar mediante los parámetros:

- **min_samples_split:** número mínimo de ejemplos en un nodo para que pueda dividirse.

- **min_samples_leaf:** número mínimo de ejemplos en un nodo hoja.

In [ ]:
## Instanciamos el modelo. Aquí sería con los parámetros por defecto, es decir,
## sin ningún tipo de restricción en la profundidad, número de ejemplos mínimos, etc.
model = DecisionTreeClassifier()

## Le pasamos al modelo el conjunto de entrenamiento para su aprendizaje
model.fit(X_train, y_train)

Para visualizar las particiones que realiza el modelo, vamos a hacer uso de la siguiente función.

In [19]:
def visualize_decision_frontiers(model, X, y, attributes_names, class_names, limits=False):
    
    plot_step = 0.02
    if not limits:
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    else:
        x_min, x_max, y_min, y_max = limits
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    
    z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    z = z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, z, cmap=plt.cm.Paired)
    
    plt.plot(X[y==0,0], X[y==0,1], 'bo', label=class_names[0])
    plt.plot(X[y==1,0], X[y==1,1], 'ro', label=class_names[1])
    
    plt.xlabel(attributes_names[0])
    plt.ylabel(attributes_names[1])
    
    plt.legend(loc=2)

In [ ]:
plt.figure(figsize=(15, 8))

limits = [ X[:,0].min()-1, X[:,0].max()+1, X[:,1].min()-1, X[:, 1].max()+1 ]

plt.subplot(1,2,1)
visualize_decision_frontiers(model, X_train, y_train,
                             attributes_names, class_names, limits=limits)
plt.title('Training')

plt.subplot(1,2,2)
visualize_decision_frontiers(model, X_test, y_test,
                             attributes_names, class_names, limits=limits)
plt.title('Test');

In [ ]:
## Calidad predictiva del modelo

print("Score training = {}".format(model.score(X_train, y_train)))
print("Score test = {}".format(model.score(X_test, y_test)))

Como se puede observar, el modelo hace excesivas divisiones para intentar separar todos los puntos, por lo tanto, está **sobreajustando** a los datos de entrenamiento. Esto hace que el modelo no sepa generalizar y las predicciones sean malas.

Para ello, hay que establecer restricciones, por ejemplo, de profundidad o número mínimo de observaciones para conseguir reglas que sean generales.

In [ ]:
## Instanciamos el modelo con determinadas restricciones
## Profundidad máxima: cuántos particiones máximas le permitimos hacer.
## Mínimo número de muestras mínima en el nodo hoja.
model = DecisionTreeClassifier(max_depth=4, min_samples_leaf=0.1)

## min_sample_leaf puede ser entero (N muestras) o real (% muestras)

## Le pasamos al modelo el conjunto de entrenamiento para su aprendizaje
model.fit(X_train, y_train)

Una vez tenemos el modelo entrenado, vamos a visualizar qué particiones ha realizado o lo que es lo mismo, las reglas.

In [ ]:
plt.figure(figsize=(20,10), dpi=150)
plot_tree(model, rounded=True, proportion=False, label='none',
          feature_names=attributes_names,
          class_names=class_names,
          filled=True);

En cada nodo el árbol muestra:

- Valor de la métrica que determina el criterio de división (por defecto, Gini).
- Número de muestras total en el nodo.
- Número de muestras clasificadas en cada clase. Aquí concretamente [N muestras clase 0, N muestras clase 1].
- Clase estimada. Calcula un umbral para determinar a qué clase pertenece.

In [ ]:
## Reglas
texto_modelo = export_text(
                    decision_tree = model,
                    feature_names = attributes_names
               )
print(texto_modelo)

Ahora vamos a ver el rendimiento del modelo tanto en entrenamiento como validación.

In [ ]:
## Calidad predictiva del modelo

print("Score training = {}".format(model.score(X_train, y_train)))
print("Score test = {}".format(model.score(X_test, y_test)))

## Equivale al accuracy
print(f"\nAccuracy en test: {accuracy_score(y_test, model.predict(X_test))}")

## Calculamos también precisión, recall y estadístico AUC
print(f"\nPrecisión en test: {precision_score(y_test, model.predict(X_test))}")
print(f"Recall en test: {recall_score(y_test, model.predict(X_test))}")
print(f"AUC en test: {roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])}")

## Matriz de confusión
print("\nMatriz de confusión en test:\n")
print(confusion_matrix(y_test, model.predict(X_test)))
# row: real class; column: predicted class

Visualizamos las particiones tanto en entrenamiento como en test para ver si hay sobreajuste.

In [ ]:
plt.figure(figsize=(15, 8))

limits = [ X[:,0].min()-1, X[:,0].max()+1, X[:,1].min()-1, X[:, 1].max()+1 ]

plt.subplot(1,2,1)
visualize_decision_frontiers(model, X_train, y_train,
                             attributes_names, class_names, limits=limits)
plt.title('Training')

plt.subplot(1,2,2)
visualize_decision_frontiers(model, X_test, y_test,
                             attributes_names, class_names, limits=limits)
plt.title('Test');

Como puede observarse, el árbol construye una separación en forma de escalera. Tiene sentido al parecer que con una línea oblicua pudieran separarse ambas clases. Sin embargo, los árboles no son capaces de trazar diagonales, de ahí que lo intente con separaciones verticales a través de las reglas.

## Explotación de una nueva observación

Imaginemos que tenemos nuevas observaciones y queremos saber su probabilidad a pertenecer a cada una de las clases. Para ello, haremos uso del modelo entrenado y lo explotaremos para estas nuevas observaciones.

In [31]:
X_new = np.array([[-15.0, -10.0], [10.0, 15.0], [0, -2.0]])

In [ ]:
X_new

In [ ]:
model.predict_proba(X_new)

## Primera columna: clase azul
## Segunda columna: clase roja

## Guardado del modelo

Si queremos usar el modelo en otra ocasión o hacer versionados, podemos guardar una copia en formato pickle.

In [34]:
import pickle

data = {
    "modelo": model,
    "comentarios": "versión del 26 de enero de 2024"
}

with open("tree.pkl", 'wb') as f:
    pickle.dump(data, f)
    
## 'wb': w indica que genere el fichero y machaque si ya existe uno previo. b indica que grabe datos en binario.

## Carga del modelo guardado

Si quisiéramos cargar un modelo que hemos guardado, debemos seguir la siguiente sentencia de código.

In [35]:
## Carga de un modelo guardado
import pickle

with open("tree.pkl", 'rb') as f:
    modelo_cargado = pickle.load(f)
    
## 'rb': r indica que lea el fichero.

In [ ]:
## Vemos qué contiene el objeto cargado
modelo_cargado

In [37]:
## Nos quedamos con el modelo para poder usarlo
modelo = modelo_cargado["modelo"]

In [ ]:
## Probamos a explotar
modelo.predict_proba(X_new)